https://wikidocs.net/45101 참고하시면 바로 이해됩니당!

#### 할일

- 텍스트를 단락으로 구분: 제가 빨리 해서 올려드릴게요!
- 특수문자, 필요없는 문자

- 맞춤법 검사 라이브러리: 유솔님
- 전치사 처리 // 형태소 토큰화 라이브러리 

- 어디까지 돌아가는지 체크: 지원님

- 다음주 수요일 모임
-----------------------

- 보고서(LSTM, RNN 모델, 파라미터 조정)
- 데모영상

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM

import io

### text 불러오기

- 단락으로 다 정리가 되어있고, 단락은 \n으로 구분되어있다고 가정,
- 맞춤법, 띄어쓰기 모두 교정되어있다고 가정

In [ ]:
filename = "[로판] 다음생에도 과로사라고요(-86).txt"
with io.open(filename) as f:
    text = f.read()

In [ ]:
print(text)

### 토큰화

- 문장을 쪼개 단어를 분리하고, 단어 각각에 대응하는 숫자를 부여

In [ ]:
tk = Tokenizer()
tk.fit_on_texts([text])

print("단어의 개수:", len(tk.word_index))

In [ ]:
print(tk.word_index)

### 인코딩, 훈련데이터 만들기

In [ ]:
traindata = []

for phrase in text.split("\n"):
    
    #단락을 숫자리스트로
    ph_splited = tk.texts_to_sequences([phrase])[0]
    
    #훈련데이터 생성 후 traindata에 append
    for i in range(1, len(ph_splited)):
        temp = []
        for j in range(0, i + 1):
            temp.append(ph_splited[j])
        traindata.append(temp)

print(traindata)

In [ ]:
print("훈련데이터 크기: %d" % len(traindata))

### 패딩넣기

In [ ]:
max_len = 0
for data in traindata:
    if (len(data) > max_len):
        max_len = len(data)

print("훈련데이터요소의 최대크기(패딩크기): %d" % max_len)

In [ ]:
traindata = pad_sequences(traindata, maxlen = max_len, padding = "pre")

In [ ]:
print(traindata)

### x 데이터, y 데이터 만들기

In [ ]:
traindata = np.array(traindata)
x = traindata[:, :max_len - 1]
y = traindata[:, max_len - 1]

In [ ]:
print(x)

In [ ]:
print(y)

In [ ]:
y = to_categorical(y, dtype = np.int64)

### 모델 만들기

In [ ]:
size = len(tk.word_index) + 1

model = Sequential()
model.add(Embedding(size, 10, input_length = max_len - 1)) # 레이블을 분리하였으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs = 100, verbose = 2, batch_size = 10)

### Generator 만들기

In [ ]:
def get_word(index, dic):
    
    for key,value in dic.items():
        if (value == index):
            return (key)

In [ ]:
def RNN_novel_generator(model, tk, prompt, num):
    for n in range(num):
        splited = tk.texts_to_sequences([prompt])[0]
        padded = pad_sequences([splited], maxlen = max_len - 1, padding = "pre")
        predicted_idx = model.predict_classes(padded, verbose = 0)
        predicted_word = get_word(predicted_idx, tk.word_index)
        
        prompt = prompt + ' ' + predicted_word
    return (prompt)

In [ ]:
print(RNN_novel_generator(model, tk, "말", 30));

### LSTM

In [ ]:
model2 = Sequential()
model2.add(Embedding(size, 10, input_length = max_len-1))
model2.add(LSTM(128))
model2.add(Dense(size, activation = 'softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(x, y, epochs = 200, verbose = 2)

In [ ]:
print(RNN_novel_generator(model, tk, "말", 30));